# Calibration History

- 2024-08-19
  - Device 01 PM2.5: 7.7 to 8.5 (+0.8, or ~10%)
  - Device 02 PM2.5: 5.3 to 5.1 (-0.2 or ~5%)

In [3]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as sm
from datetime import datetime, timedelta
from functools import reduce

In [4]:
start_incl = datetime.fromisoformat("2024-08-18T17:00:00+08.00")
end_excl = datetime.fromisoformat("2024-08-19T11:00:00+08.00")
times = [start_incl + timedelta(hours=h) for h in range(int((end_excl - start_incl).total_seconds() / 60 / 60))]

def calculate_adjustment(df: pd.DataFrame, dst: str, src: str):
    regression = sm.ols(formula=f"{dst} ~ {src} -1", data=df).fit()
    coefficient = regression.params.values[0]
    max_src = df[src].max()
    print(f"{dst} = {src} × {coefficient:.3f} (r²={regression.rsquared:.3f})")
    ax = df.plot.scatter(dst, src)
    ax.plot([0, max_src * coefficient], [0, max_src], color="0.9", linestyle="dotted")

In [ ]:
%run "Common.ipynb"

In [ ]:
dfs = [grp.reset_index(level=0, drop=True).rename(columns={"PM2.5(ug/m³)": f"Device{dev}"}) for dev, grp in df.loc[(slice(None), times), ["PM2.5(ug/m³)"]].groupby(level=0)]
joined_df = reduce(lambda l, r: l.join(r), dfs)

In [ ]:
_ = joined_df.plot()
calculate_adjustment(joined_df, "Device03", "Device01")
calculate_adjustment(joined_df, "Device03", "Device02")